In [1]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import sys
sys.path.append(r'D:\kasia\tracking\E6_exp\code\tracks_interactions')
from tracks_interactions.db.db_model import NO_PARENT, CellDB, TrackDB
import dask.array as da
import napari
import numpy as np
import os
os.environ['NAPARI_PERFMON'] = '0'
import cv2
from readlif.reader import LifFile

In [2]:
exp_path = r'R:\Dante\hillman_academy_2023\Jinny_test.lif'
exp = LifFile(exp_path)

In [3]:
sel_name = 'TileScan 5/E6_Merged'
t = [image for image in exp.get_iter_image() if sel_name in image.name][0]
t

'LifImage object with dimensions: Dims(x=16802, y=16793, z=1, t=241, m=1)'

In [4]:
t.scale

(3.076923076923077, 3.0769230769230766, None, 0.0005601651180046172)

In [5]:
# specify parameters

cell_rad = 80

# timer
interval = 30
# timer position
timer_r = 20
timer_c = 140

text_parameters = {
    'string': '{time}',
    'size': 18,
    'color': 'white',
    'anchor': 'upper_right',
    'translation': [-3, 0]
}

# scale bar parameters
# add scale bar
height_px = 1
pos_col = 10
pos_row = 150
len_um = 20
len_pixel = t.scale[0] * len_um / 2 # because we were using the second resolution level
scale_bar_text = f'{len_um} \u03BCm'

save_dir = r'D:\kasia\ovcar_movies'
video_speed = 15

In [6]:
# get images
ch0_path = r'D:\kasia\tracking\E6_exp\E6_C0.zarr'
ch1_path = r'D:\kasia\tracking\E6_exp\E6_C1.zarr'

ch0=da.from_zarr(ch0_path,1)
ch1=da.from_zarr(ch1_path,1)

In [7]:
# get access to the database
db_path = r'D:\kasia\tracking\E6_exp\double_segmentation_ultrack\Exp6_gardener_example_to_process.db'
engine = create_engine(f'sqlite:///{db_path}')
session = sessionmaker(bind=engine)()

from sqlalchemy import text
session.execute(text("ANALYZE;"))
session.commit()

In [8]:
t=session.query(TrackDB).filter(TrackDB.accepted_tag == True).all()

In [9]:
def cut_cell():

    ch0_cell = []
    ch1_cell = []

    r_rad = c_rad = cell_rad
    for cell in cells:

        frame0 = ch0[cell.t,cell.row-r_rad:cell.row+r_rad, cell.col-c_rad:cell.col+c_rad]
        ch0_cell.append(frame0)
        frame1 = ch1[cell.t,cell.row-r_rad:cell.row+r_rad, cell.col-c_rad:cell.col+c_rad]
        ch1_cell.append(frame1)

    ch0_cell = np.array(ch0_cell)
    ch1_cell = np.array(ch1_cell)
    
    return ch0_cell, ch1_cell

In [10]:
def generate_shapes_and_annotations():

    # generate text for annotations
    test_texts = []
    ss = []
    for i,t in enumerate(range(ch0_cell.shape[0])):

        test_text = f'{str(int(t*interval/60)).zfill(2)}:{str((t*interval)%60).zfill(2)} h:min'
        test_texts.append(test_text)

        s = np.array([[i,timer_r,timer_r],[i,timer_r,timer_c],[i,timer_c,timer_c],[i,timer_c,timer_r]])
        ss.append(s)

    return test_texts, ss

In [11]:
def add_shapes():

    global viewer

    shapes_layer = viewer.add_shapes(
        ss,
        face_color='transparent',
        edge_color='transparent',
        text=text_parameters,
        properties={'time': test_texts},
        name='bounding box',
    )

In [12]:
def add_scale_bar():
    
    global viewer

    scale_bar = viewer.add_shapes(
        [[pos_row+height_px, pos_col], [pos_row+height_px, pos_col+len_pixel], [pos_row, pos_col+len_pixel], [pos_row, pos_col]],
        edge_color='white',
        edge_width=0,
        face_color = 'white',
        name='scale bar',
        text = {'text': '{len}', 'color': 'white', 'size': 18, 'anchor': 'upper_left','translation': [-1, 5]},
        properties={'len': scale_bar_text}
    )

In [13]:
def create_screenshots():

    l = []
    for n in range(ch0_cell.shape[0]):
        viewer.dims.set_current_step(0,n)
        screenshot = viewer.screenshot()
        l.append(screenshot[:,:,:-1])

    return l

In [14]:
for cell_num in range(len(t)):

    # find cell id
    track_id = t[cell_num].track_id

    # get all cells from the track
    cells=session.query(CellDB).filter_by(track_id = track_id).order_by(CellDB.t).all()

    tag = ''
    for cell in cells:
        if cell.tags.get('apoptosis', False):
            tag='_apo'
            break
        if cell.tags.get('mitosis', False):
            tag='_mit'
            break

    # cut the cell
    ch0_cell, ch1_cell = cut_cell()

    # create napari viewer
    viewer = napari.Viewer()    
    viewer.add_image(ch0_cell, name='ch0',colormap='green',contrast_limits=[0,4000],blending='additive')
    #viewer.add_image(ch1_cell, name='ch1',colormap='magenta',contrast_limits=[0,3000],blending='additive')

    # generate shapes and annotations
    test_texts, ss = generate_shapes_and_annotations()

    # add shapes
    add_shapes()

    #add scale bar
    add_scale_bar()

    # get screenshots
    l = create_screenshots()

    # save video
    save_path = os.path.join(save_dir, f'track_{cells[-1].track_id}{tag}.avi')

    fourcc = cv2.VideoWriter_fourcc(*'XVID')
    out = cv2.VideoWriter(save_path, fourcc, video_speed, (l[0].shape[1],l[0].shape[0]))

    for frame in l:
        out.write(frame)

    out.release()

    viewer.close()